In [ ]:
%matplotlib inline
import numpy as np
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
import os
from glob import glob
import torch
from tqdm import tqdm

from EuroSAT_dataset import EuroSAT
from network import Net


%load_ext autoreload
%autoreload 2


if not os.path.exists('./outputs'):
    os.mkdir('./outputs')

print('Imports OK')

In [ ]:
# Global parameters

# If USE_CUDA is True, computations will be done using the GPU (may not work in all systems)
# This will make the calculations happen faster
USE_CUDA = 'TODO'

DATASET_PATH = './EuroSAT_data'

BATCH_SIZE = 'TODO' # Number of images that are used for calculating gradients at each step

NUM_EPOCHS = 'TODO' # Number of times we will go through all the training images. Do not go over 25

LEARNING_RATE = 'TODO' # Controls the step size
MOMENTUM = 'TODO' # Momentum for the gradient descent
WEIGHT_DECAY = 'TODO' # Regularization factor to reduce overfitting

    

print('Parameters OK')

In [ ]:
# Create datasets and data loaders
train_dataset = 'TODO'
train_loader = 'TODO'

test_dataset = 'TODO'
test_loader = 'TODO'


print('Dataloaders OK')

In [ ]:
# Create network
'TODO'
#if USE_CUDA:
'TODO'

print('Network OK')

In [ ]:
# Define criterion, optimizer, and scheduler

criterion = 'TODO'
optimizer = 'TODO'
scheduler = 'TODO'

In [ ]:
# Helper function to organize main loop
# This function is called for training and for testing at each epoch

def run_epoch(net, optimizer, dataloader, criterion, train=True, cuda=USE_CUDA):
    epoch_total_loss = 0
    epoch_total_samples = 0
    epoch_total_correct = 0

    for sample in tqdm(dataloader):
        img = 'TODO'
        label = 'TODO'

        if cuda:
            'TODO'

        if train: #Be aware that this function should work both for training and testing
            'TODO'
        
        'TODO'
        
        with torch.no_grad():
            epoch_total_samples += 'TODO'
            epoch_total_loss += 'TODO'
            epoch_total_correct += 'TODO'

    return epoch_total_loss / epoch_total_samples, epoch_total_correct / epoch_total_samples

In [ ]:
# Main loop

train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
epochs = []

for epoch in range(1, NUM_EPOCHS+1):
    print(f'\n\nRunning epoch {epoch} of {NUM_EPOCHS}...\n')
    epochs.append(epoch)

    #-------------------------Train-------------------------
    # Set network train/test mode to 'train' in the next line
    'TODO'
    
    loss, accuracy = 'TODO'
    train_loss.append(loss.cpu())
    train_accuracy.append(accuracy.cpu())
    
    # Update learning rate with scheduler in the next line
    'TODO'
    
    

    #-------------------------Test--------------------------
    # Set network train/test mode to 'test' in the next line
    'TODO'
    
    with torch.no_grad():
        loss, accuracy = 'TODO'
    test_loss.append(loss.cpu())
    test_accuracy.append(accuracy.cpu())
    print(f'\nEpoch {epoch} validation results: Loss={loss.cpu()} | Accuracy={accuracy.cpu()}\n')


    
    # Plot and save
    plt.figure(figsize=(12, 8), num=1)
    plt.clf()
    plt.plot(epochs, train_loss, label='Train')
    plt.plot(epochs, test_loss, label='Test')
    plt.legend()
    plt.grid()
    plt.title('Cross entropy loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('outputs/01-loss.pdf')

    plt.figure(figsize=(12, 8), num=2)
    plt.clf()
    plt.plot(epochs, train_accuracy, label='Train')
    plt.plot(epochs, test_accuracy, label='Test')
    plt.legend()
    plt.grid()
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig('outputs/02-accuracy.pdf')

In [ ]:
print(f'Final train loss: {train_loss[-1]}')
print(f'Final test loss: {test_loss[-1]}')
print(f'Final train accuracy: {train_accuracy[-1]}')
print(f'Final test accuracy: {test_accuracy[-1]}')